<a href="https://colab.research.google.com/github/lalanglarano/TAKLUBAN-FILIPINO-NATIVE-LANGUAGE-PROFANE-DETECTION/blob/old-main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch

In [2]:
!pip install transformers

In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

In [5]:
# Load dataset from CSV files
dataset = load_dataset('csv', data_files={
    'train': '/content/datasets/BERT_TRAIN.csv',
    'validation': '/content/datasets/BERT_VAL.csv'
})

# Load pre-trained BERT tokenizer and model for sequence classification (2 output classes)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenization function for dataset
def tokenize_function(examples):
    # Tokenize the input text
    tokens = tokenizer(
        examples['sentence'],
        padding='max_length',
        max_length=128,
        truncation=True
    )
    # Add labels to the tokenized output
    tokens['labels'] = examples['profane']
    return tokens

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Set format for PyTorch tensors
tokenized_datasets.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

# Training Arguments base on BERT paper
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=0.00001,  # 1e-5
    per_device_train_batch_size=64,  # Batch size
    per_device_eval_batch_size=64,
    num_train_epochs=3,  # Number of epochs
    weight_decay=0.01,
    logging_dir='./logs',
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

# Train the model
trainer.train()

# Save the model
trainer.save_model('./trained_bert')

# Evaluate the model
results = trainer.evaluate()

# Print results
print(results)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,0.126579
2,No log,0.092139
3,0.127700,0.089167


{'eval_loss': 0.08916709572076797, 'eval_runtime': 9.5343, 'eval_samples_per_second': 144.532, 'eval_steps_per_second': 2.307, 'epoch': 3.0}


In [6]:
!pip install safetensors

In [7]:
from safetensors.torch import load_file
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [8]:
# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load model configuration and weights
model = BertForSequenceClassification.from_pretrained(
    './trained_bert/',
    config='./trained_bert/config.json',
    from_tf=False
)

# Load the weights from safetensors format
state_dict = load_file("./trained_bert/model.safetensors")
model.load_state_dict(state_dict)

# Set the model to evaluation mode
model.eval()

# Initialize lists to store true labels and predictions for metrics calculation
true_labels = []
predictions = []

# Loop for user input
while True:
    # Ask user for a sentence
    sentence = input("Enter a sentence to classify or type 'exit' to stop: ")

    # Exit condition
    if sentence.lower() == 'exit':
        break

    # User input: tokenize and make prediction
    inputs = tokenizer(sentence, return_tensors="pt")

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Get the predicted class (0: Not Profane, 1: Profane)
    predicted_class = torch.argmax(logits, dim=-1).item()

    # Simulate true label for demonstration purposes (You can replace this with actual labels if available)
    true_label = int(input("Enter the true label (0 for Not Profane, 1 for Profane): "))

    # Append to the list
    true_labels.append(true_label)
    predictions.append(predicted_class)

    # Print the result
    if predicted_class == 1:
        print("The sentence is classified as: Profane")
    else:
        print("The sentence is classified as: Not Profane")

# Calculate and display evaluation metrics
if true_labels and predictions:
    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')

    print("\nEvaluation Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
else:
    print("No sentences classified, no metrics to display.")

Enter a sentence to classify or type 'exit' to stop: masaya doon sa labasan maraming tinda
Enter the true label (0 for Not Profane, 1 for Profane): 0
The sentence is classified as: Not Profane
Enter a sentence to classify or type 'exit' to stop: napaka ganda ng inyong kasuotan mga binibini
Enter the true label (0 for Not Profane, 1 for Profane): 0
The sentence is classified as: Not Profane
Enter a sentence to classify or type 'exit' to stop: exit

Evaluation Metrics:
Accuracy: 1.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
